In [1]:
import cv2
import numpy as np
import mediapipe as mp

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [29]:
def get_landmarks(multi_hand_landmarks):
    '''
    将mediapipe的识别结果转换为列表形式方便操作
    multi_hand_landmarks:多手坐标results.multi_hand_landmarks
    '''
    land_marks = []
    for hand_landmarks in multi_hand_landmarks:
        tmp = str(hand_landmarks).replace('\n', '').replace('{', '').replace('landmark  ', '').replace('}', ' ').replace('  ', ' ').replace(':', '')[1:-1].split(' ')
        land_marks.append([(float(tmp[i+1]), float(tmp[i+3]), float(tmp[i+5])) for i in range(0, len(tmp), 6)])
    return land_marks

def get_label(multi_handedness):
    '''
    得到mediapipe对左右手的识别结果
    multi_handedness:识别标签results.multi_handedness
    '''
    hands_info = {}
    index      = 0
    for hand_info in multi_handedness:
        hand_info = str(hand_info).replace('\n', '').replace('  ',' ').replace('"', '').replace('}', '')
        hand_info = hand_info.split(' ')
        hands_info[hand_info[7]] = (index, float(hand_info[5]))
        index += 1
    return hands_info

In [28]:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue
        # 取消图像标记可提升程序性能
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # 将图片翻转
        image = cv2.flip(image, 1)
        # 进行预测
        results = hands.process(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.multi_hand_landmarks != None:
            # 绘制手部关键点图像
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style())
            hand_points = get_landmarks(results.multi_hand_landmarks)
            hands_info  = get_label(results.multi_handedness)
            for hand_label in hands_info.keys():
                # 若只出现一只手，则不使用索引
                hand_index = 0
                if len(hand_points) > 1:
                    # 若只出现多只手，则使用索引
                    hand_index = hands_info[hand_label][0]
                tmp_point = (
                    int(hand_points[hand_index][0][0]*image.shape[1]),
                    int(hand_points[hand_index][0][1]*image.shape[0])
                )
                # 绘制左右手的标签、
                image = cv2.putText(image, hand_label, tmp_point, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.imshow('MediaPipe Hands', image)
        if cv2.waitKey(5) & 0xFF == 27:
            break
cap.release()
cv2.destroyAllWindows()

In [132]:
cap.release()

In [25]:
results.multi_handedness, hands_info

([classification {
    index: 1
    score: 0.9591565728187561
    label: "Right"
  },
  classification {
    index: 0
    score: 0.9091022610664368
    label: "Left"
  }],
 {'Right': (1, 0.9591565728187561), 'Left': (0, 0.9091022610664368)})

In [117]:
get_landmarks(results.multi_hand_landmarks)[0]

[('0.41970664262771606', '0.6594709157943726', '8.223367764514933e-09'),
 ('0.48435455560684204', '0.6475182175636292', '-0.025378096848726273'),
 ('0.5339502692222595', '0.6051388382911682', '-0.03971827030181885'),
 ('0.5692800879478455', '0.5667318105697632', '-0.054267965257167816'),
 ('0.6048367619514465', '0.5476455688476562', '-0.07062353193759918'),
 ('0.4957050681114197', '0.49334418773651123', '-0.016261879354715347'),
 ('0.5225257873535156', '0.41995614767074585', '-0.032016821205616'),
 ('0.539300799369812', '0.37440162897109985', '-0.04613199084997177'),
 ('0.551587700843811', '0.33479446172714233', '-0.05693177878856659'),
 ('0.4605002999305725', '0.4735385775566101', '-0.018184583634138107'),
 ('0.4744134247303009', '0.3809613585472107', '-0.031147565692663193'),
 ('0.48443475365638733', '0.3220056891441345', '-0.04920611530542374'),
 ('0.490028977394104', '0.27200847864151', '-0.06422082334756851'),
 ('0.4258636236190796', '0.47305160760879517', '-0.024022875353693962')

In [108]:
len([(a[i+1], a[i+3], a[i+5]) for i in range(0, len(a), 6)])

21

In [118]:
hands_info

{'Left': (0, 0.8885172009468079)}